<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте простое, сложное и множественное наследование

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [10]:
using System;
using System.Collections.Generic;

// Базовый класс с новыми атрибутами
public class BankAccount
{
    public string AccountNumber { get; set; }
    public decimal Balance { get; set; }
    public string AccountType { get; set; }
    public DateTime OpeningDate { get; set; } // Новый атрибут: дата открытия
    public string OwnerName { get; set; } // Новый атрибут: владелец счета
    public string Currency { get; set; } // Новый атрибут: валюта счета
    public bool IsActive { get; set; } // Новый атрибут: статус счета

    public BankAccount(string accountNumber, decimal balance, string accountType, string ownerName, string currency = "RUB")
    {
        AccountNumber = accountNumber;
        Balance = balance;
        AccountType = accountType;
        OwnerName = ownerName;
        Currency = currency;
        OpeningDate = DateTime.Now;
        IsActive = true;
    }

    public virtual string GetInfo()
    {
        string status = IsActive ? "Активный" : "Закрыт";
        return $"Счет: {AccountNumber} | Тип: {AccountType} | Баланс: {Balance} {Currency} | Владелец: {OwnerName} | Статус: {status}";
    }

    public virtual void Deposit(decimal amount, string dataRefill)
    {
        if (!IsActive)
        {
            Console.WriteLine("Ошибка: счет закрыт");
            return;
        }

        if (amount <= 0)
        {
            Console.WriteLine("Сумма для пополнения должна быть положительной");
        }
        else
        {
            Balance += amount;
            Console.WriteLine($"Внесена сумма: {amount} | Текущий баланс: {Balance} {Currency} | Дата пополнения: {dataRefill}");
        }
    }

    public virtual void Withdraw(decimal amount)
    {
        if (!IsActive)
        {
            Console.WriteLine("Ошибка: счет закрыт");
            return;
        }

        if (amount > Balance)
        {
            Console.WriteLine("Сумма для снятия больше баланса");
        }
        else
        {
            Balance -= amount;
            Console.WriteLine($"Снята сумма: {amount} | Текущий баланс: {Balance} {Currency}");
        }
    }

    // Новый метод: перевод в другую валюту
    public virtual decimal ConvertBalance(string targetCurrency, decimal exchangeRate)
    {
        decimal convertedBalance = Balance * exchangeRate;
        Console.WriteLine($"Конвертация: {Balance} {Currency} -> {convertedBalance} {targetCurrency} (курс: {exchangeRate})");
        return convertedBalance;
    }

    // Новый метод: закрытие счета
    public virtual void CloseAccount()
    {
        if (Balance != 0)
        {
            Console.WriteLine("Нельзя закрыть счет с ненулевым балансом");
            return;
        }
        IsActive = false;
        Console.WriteLine("Счет успешно закрыт");
    }

    
    
}

// Сберегательный счет
public class SavingsAccount : BankAccount
{
    public decimal InterestRate { get; set; } //процентная ставка
    public decimal MinimumBalance { get; set; } // Новый атрибут: минимальный баланс
    public DateTime LastInterestDate { get; set; } // Новый атрибут: дата последнего начисления процентов

    public SavingsAccount(string accountNumber, decimal balance, decimal interestRate, string ownerName, decimal minimumBalance = 1000) : base(accountNumber, balance, "Сберегательный", ownerName)
    {
        InterestRate = interestRate;
        MinimumBalance = minimumBalance;
        LastInterestDate = DateTime.Now;
    }

    public override void Deposit(decimal amount, string dataRefill)
    {
        if (!IsActive)
        {
            Console.WriteLine("Ошибка: счет закрыт");
            return;
        }

        if (amount <= 0)
        {
            Console.WriteLine("Сумма для пополнения должна быть положительной");
        }
        else
        {
            decimal interest = amount * (InterestRate / 100);
            decimal finalAmount = amount + interest;

            Balance += finalAmount;
            Console.WriteLine($"Пополнение на {amount} + проценты {interest} = {finalAmount}");
            Console.WriteLine($"Текущий баланс: {Balance} {Currency} | Дата пополнения: {dataRefill}");
        }
    }

    // Новый метод: начисление ежемесячных процентов
    public void ApplyMonthlyInterest()
    {
        if (Balance >= MinimumBalance)
        {
            decimal interest = Balance * (InterestRate / 100 / 12);
            Balance += interest;
            LastInterestDate = DateTime.Now;
            Console.WriteLine($"Начислены проценты: {interest} {Currency} | Новый баланс: {Balance} {Currency}");
        }
    }

    // Новый метод: проверка соблюдения минимального баланса
    public bool CheckMinimumBalance()
    {
        bool isOk = Balance >= MinimumBalance;
        Console.WriteLine($"Минимальный баланс: {MinimumBalance} | Текущий: {Balance} | Соответствие: {(isOk ? "Да" : "Нет")}");
        return isOk;
    }
}

// Текущий счет 
public class CheckingAccount : BankAccount
{
    public decimal OverdraftLimit { get; set; }
    public int MonthlyFreeTransactions { get; set; } // Новый атрибут: бесплатные операции в месяц
    public decimal TransactionFee { get; set; } // Новый атрибут: комиссия за операцию
    private int _transactionsThisMonth; // Новый атрибут: счетчик операций

    public CheckingAccount(string accountNumber, decimal balance, decimal overdraftLimit, string ownerName, int monthlyFreeTransactions = 5, decimal transactionFee = 10) 
        : base(accountNumber, balance, "Текущий", ownerName)
    {
        OverdraftLimit = overdraftLimit;
        MonthlyFreeTransactions = monthlyFreeTransactions;
        TransactionFee = transactionFee;
        _transactionsThisMonth = 0;
    }

    public override void Withdraw(decimal amount)
    {
        if (!IsActive)
        {
            Console.WriteLine("Ошибка: счет закрыт");
            return;
        }

        if (amount <= 0)
        {
            Console.WriteLine("Сумма для снятия должна быть положительной");
            return;
        }

        // Проверка комиссии
        decimal totalAmount = amount;
        if (_transactionsThisMonth >= MonthlyFreeTransactions)
        {
            totalAmount += TransactionFee;
            Console.WriteLine($"Взимается комиссия: {TransactionFee} {Currency}");
        }

        if (OverdraftLimit + Balance >= totalAmount)
        {
            Balance -= totalAmount;
            _transactionsThisMonth++;
            Console.WriteLine($"Снято: {amount} | Новый баланс: {Balance} {Currency}");
            if (Balance < 0)
            {
                Console.WriteLine($"Внимание, использован овердрафт! Долг: {Balance} {Currency}");
            }
        }
        else
        {
            Console.WriteLine($"Превышен лимит овердрафта! Максимальная сумма для снятия: {Balance + OverdraftLimit} {Currency}");
        }
    }

    // Новый метод: сброс счетчика операций (в начале месяца)
    public void ResetMonthlyTransactions()
    {
        _transactionsThisMonth = 0;
        Console.WriteLine("Счетчик операций за месяц сброшен");
    }

    // Новый метод: информация о доступных бесплатных операциях
    public void ShowTransactionInfo()
    {
        int remaining = Math.Max(0, MonthlyFreeTransactions - _transactionsThisMonth);
        Console.WriteLine($"Бесплатных операций осталось: {remaining} из {MonthlyFreeTransactions}");
    }
}

// Инвестиционный счет
public class InvestmentAccount : BankAccount
{
    public List<string> AssetsList { get; set; }
    private List<string> DataAssetsList { get; set; }
    public string RiskLevel { get; set; } // Новый атрибут: уровень риска
    public decimal ManagementFee { get; set; } // Новый атрибут: комиссия за управление

    public InvestmentAccount(string accountNumber, decimal balance, string ownerName, string riskLevel = "Средний", decimal managementFee = 0.5m) 
        : base(accountNumber, balance, "Инвестиционный", ownerName)
    {
        AssetsList = new List<string>();
        DataAssetsList = new List<string>();
        RiskLevel = riskLevel;
        ManagementFee = managementFee;
    }

    public void AddAsset(string assetName, string dataAssert)
    {
        AssetsList.Add(assetName);
        DataAssetsList.Add(dataAssert);
        Console.WriteLine($"Добавлен актив: {assetName} | Дата: {dataAssert}");
    }

    // Новый метод: удаление актива
    public void RemoveAsset(string assetName)
    {
        int index = AssetsList.IndexOf(assetName);
        if (index != -1)
        {
            AssetsList.RemoveAt(index);
            DataAssetsList.RemoveAt(index);
            Console.WriteLine($"Актив {assetName} удален");
        }
        else
        {
            Console.WriteLine($"Актив {assetName} не найден");
        }
    }

    // Новый метод: списание комиссии за управление
    public void ChargeManagementFee()
    {
        decimal fee = Balance * (ManagementFee / 100);
        Balance -= fee;
        Console.WriteLine($"Списана комиссия за управление: {fee} {Currency}");
    }

    public override string GetInfo()
    {
        string baseInfo = base.GetInfo();
        string assetsInfo = AssetsList.Count > 0 ? $" | Активы: {string.Join(", ", AssetsList)}" : " | Активы: нет";
        return baseInfo + assetsInfo + $" \n| Уровень риска: {RiskLevel}";
    }
}

// ПРОСТОЕ НАСЛЕДОВАНИЕ: Премиум счет наследуется от Сберегательного
public class PremiumSavingsAccount : SavingsAccount
{
    public decimal BonusRate { get; set; } // Дополнительный бонусный процент
    public string CardType { get; set; } // Тип карты

    public PremiumSavingsAccount(string accountNumber, decimal balance, decimal interestRate,
                                string ownerName, decimal bonusRate, string cardType = "Platinum")
        : base(accountNumber, balance, interestRate, ownerName)
    {
        BonusRate = bonusRate;
        CardType = cardType;
        AccountType = "Премиум-сберегательный";
    }

    // Новый метод: начисление бонуса
    public void ApplyBonus()
    {
        decimal bonus = Balance * (BonusRate / 100);
        Balance += bonus;
        Console.WriteLine($"Начислен бонус: {bonus} {Currency} | Тип карты: {CardType}");
    }

    public override string GetInfo()
    {
        return base.GetInfo() + $" | Бонусная ставка: {BonusRate}% | Карта: {CardType}";
    }
}

// СЛОЖНОЕ НАСЛЕДОВАНИЕ: Мультивалютный счет
public class MultiCurrencyAccount : BankAccount
{
    //Dictionary - это коллекция пар "ключ-значение"
    public Dictionary<string, decimal> CurrencyBalances { get; set; } // Балансы в разных валютах

    public MultiCurrencyAccount(string accountNumber, decimal initialBalance, string ownerName)
        : base(accountNumber, initialBalance, "Мультивалютный", ownerName, "MULTI")
    {
        CurrencyBalances = new Dictionary<string, decimal>
        {
            { "RUB", initialBalance },
            { "USD", 0 },
            { "EUR", 0 }
        };
    }

    // Новый метод: пополнение в конкретной валюте
    public void DepositInCurrency(decimal amount, string currency, string dataRefill)
    {
        if (CurrencyBalances.ContainsKey(currency)) //ContainsKey - метод Dictionary, который проверяет существует ли указанный ключ в словаре
        {
            CurrencyBalances[currency] += amount;
            Console.WriteLine($"Внесено {amount} {currency} | Дата: {dataRefill}");
            Console.WriteLine($"Валюта: {currency} | Сумма: {CurrencyBalances[currency]}");
        }
        else
        {
            Console.WriteLine($"Валюта {currency} не поддерживается");
        }
    }

    // Переопределение базовой валюты для работы с основной RUB
    public override void Deposit(decimal amount, string dataRefill)
    {
        DepositInCurrency(amount, "RUB", dataRefill);
    }

}


// МНОЖЕСТВЕННОЕ НАСЛЕДОВАНИЕ (через интерфейсы)

// Интерфейс для счетов с уведомлениями
public interface INotifiable
{
    void SendNotification(string message); //метод для отправки сообщений пользователю 
    string Email { get; set; }
    string Phone { get; set; }
}

// Интерфейс для счетов с онлайн-операциями
public interface IOnlineOperable
{
    void OnlineTransfer(string toAccount, decimal amount); //перевод на счет
    bool IsOnlineEnabled { get; set; } //есть ли онлайн-доступ
}

// Класс
public class DigitalAccount : CheckingAccount, INotifiable, IOnlineOperable
{
    public string Email { get; set; }
    public string Phone { get; set; }
    public bool IsOnlineEnabled { get; set; }
    public string AppVersion { get; set; }

    public DigitalAccount(string accountNumber, decimal balance, decimal overdraftLimit, string ownerName, string email, string phone)
        : base(accountNumber, balance, overdraftLimit, ownerName)
    {
        Email = email;
        Phone = phone;
        IsOnlineEnabled = true;
        AppVersion = "2.0";
        AccountType = "Цифровой";
    }

    // Реализация INotifiable
    public void SendNotification(string message)
    {
        Console.WriteLine($"Отправка уведомления: {message}");
        Console.WriteLine($"Email: {Email} | SMS на номер: {Phone}");
    }

    // Реализация IOnlineOperable
    public void OnlineTransfer(string toAccount, decimal amount)
    {
        if (!IsOnlineEnabled)
        {
            Console.WriteLine("Онлайн-операции отключены");
            return;
        }

        if (amount > Balance)
        {
            Console.WriteLine("Недостаточно средств для перевода");
            return;
        }

        Balance -= amount;
        SendNotification($"Онлайн-перевод {amount} {Currency} на счет {toAccount}");
        Console.WriteLine($"Перевод {amount} {Currency} на счет {toAccount} выполнен");
        Console.WriteLine($"Оставшаяся сумма: {Balance}");
    }

    public override string GetInfo()
    {
        return base.GetInfo() + $" | Email: {Email} | Онлайн-доступ: {(IsOnlineEnabled ? "Да" : "Нет")}";
    }
}

//ВЫВОД
Console.WriteLine("          БАЗОВЫЙ СЧЕТ           ");

BankAccount bn = new BankAccount("BN125869", 50000, "Базовый", "Сидоров Иван");
bn.Deposit(20000, "29.02.2024");
bn.Withdraw(10000);
bn.ConvertBalance("BYN", 0.038m);
bn.CloseAccount();
Console.WriteLine(bn.GetInfo());  

Console.WriteLine("");

Console.WriteLine("          СБЕРЕГАТЕЛЬНЫЙ СЧЕТ           ");
SavingsAccount sa = new SavingsAccount("SA665655", 80000, 15, "Бусько Дарья", 10000);
sa.Deposit(-20000, "12.12.2012");
sa.ApplyMonthlyInterest();
sa.CheckMinimumBalance();

Console.WriteLine("");

Console.WriteLine("          ТЕКУЩИЙ СЧЕТ           ");
CheckingAccount ca = new CheckingAccount("CA6165536", 9500, 30000, "Козлова Марина");
ca.Withdraw(80000);
ca.ShowTransactionInfo();

Console.WriteLine("");

Console.WriteLine("          ИНВЕСТИЦИОННЫЙ СЧЕТ           ");
InvestmentAccount ia = new InvestmentAccount("IA32356", 65000, "Рандымова Анна", "Высокий", 1.0m);
ia.AddAsset("Газпром ", "30.02.1900");
ia.AddAsset("Лукойл  ", "01.01.2000");
ia.AddAsset("Роснефть", "12.12.2012");
Console.WriteLine(ia.GetInfo());
ia.ChargeManagementFee();
ia.RemoveAsset("Газпром ");
Console.WriteLine(ia.GetInfo());

Console.WriteLine("");

Console.WriteLine("          ПРЕМИУМ СЧЕТ           ");
PremiumSavingsAccount psa = new PremiumSavingsAccount("PREMIUM001", 100000, 15, "Новикова Ольга", 5, "Diamond");
Console.WriteLine(psa.GetInfo());
psa.ApplyBonus();
psa.ApplyMonthlyInterest();

Console.WriteLine("");

Console.WriteLine("          МУЛЬТИВАЛЮТНЫЙ  СЧЕТ           ");
MultiCurrencyAccount mca = new MultiCurrencyAccount("MULTI001", 50000, "Дмитрий Смирнов");
Console.WriteLine(mca.GetInfo());
mca.DepositInCurrency(1000, "USD", "20.09.2025");
mca.DepositInCurrency(500, "EUR", "20.09.2025");
Console.WriteLine(mca.GetInfo());

Console.WriteLine("");

Console.WriteLine("          ЦИФРОВОЙ  СЧЕТ           ");
DigitalAccount da = new DigitalAccount("DIGITAL001", 75000, 20000, "Сергей Кузнецов", "sergey@mail.ru", "+79161234567");
Console.WriteLine(da.GetInfo());
da.OnlineTransfer("RECIPIENT123", 5000);
da.SendNotification("Добро пожаловать в наш банк!");











          БАЗОВЫЙ СЧЕТ           
Внесена сумма: 20000 | Текущий баланс: 70000 RUB | Дата пополнения: 29.02.2024
Снята сумма: 10000 | Текущий баланс: 60000 RUB
Конвертация: 60000 RUB -> 2280,000 BYN (курс: 0,038)
Нельзя закрыть счет с ненулевым балансом
Счет: BN125869 | Тип: Базовый | Баланс: 60000 RUB | Владелец: Сидоров Иван | Статус: Активный

          СБЕРЕГАТЕЛЬНЫЙ СЧЕТ           
Сумма для пополнения должна быть положительной
Начислены проценты: 1000,0000 RUB | Новый баланс: 81000,0000 RUB
Минимальный баланс: 10000 | Текущий: 81000,0000 | Соответствие: Да

          ТЕКУЩИЙ СЧЕТ           
Превышен лимит овердрафта! Максимальная сумма для снятия: 39500 RUB
Бесплатных операций осталось: 5 из 5

          ИНВЕСТИЦИОННЫЙ СЧЕТ           
Добавлен актив: Газпром  | Дата: 30.02.1900
Добавлен актив: Лукойл   | Дата: 01.01.2000
Добавлен актив: Роснефть | Дата: 12.12.2012
Счет: IA32356 | Тип: Инвестиционный | Баланс: 65000 RUB | Владелец: Рандымова Анна | Статус: Активный | Активы: Газпр